In [2]:
import pandas as pd

# Đọc file dữ liệu 'ptttkhdl-data.csv' chứa các trường roles, companies, locations, experience, skills
data = pd.read_csv('ptttkhdl-data.csv')
print(data.head())


   Unnamed: 0                                              roles  \
0           0                          Data Scientist, Marketing   
1           1  Hexaware_ Looking Immediate joiners -Data Scie...   
2           2         Data Scientist-Senior Associate - P&T Labs   
3           3         Data Scientist - CROCI Investment Strategy   
4           4                                    Adobe Analytics   

                     companies  \
0                     Coursera   
1        Hexaware Technologies   
2  PwC Service Delivery Center   
3                Deutsche Bank   
4  PwC Service Delivery Center   

                                           locations experience  \
0  Kolkata, Mumbai, New Delhi, Hyderabad/Secunder...    4-6 Yrs   
1                Hybrid - Navi Mumbai, Pune, Chennai    4-9 Yrs   
2                       Mumbai, Hyderabad, Bengaluru    4-6 Yrs   
3                                             Mumbai    3-5 Yrs   
4                                    Bangalore Rura

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Kết hợp các trường vào một cột để tạo ma trận TF-IDF
data['kethop'] = data['roles'] + " " + data['companies'] + " " + data['locations'] + " " + data['experience'].astype(str) + " " + data['skills']

# Chuyển đổi văn bản thành ma trận TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data['kethop'])

# Tạo mô hình KNN
model = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(tfidf_matrix)

def recommend_jobs(query):
    query_vec = vectorizer.transform([query])
    distances, indices = model.kneighbors(query_vec)
    return data.iloc[indices[0]]


In [4]:
import tkinter as tk
from tkinter import ttk

class JobRecommenderApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Đề xuất công việc")

        # Tạo các trường nhập liệu
        self.roles_var = tk.StringVar()
        self.companies_var = tk.StringVar()
        self.locations_var = tk.StringVar()
        self.experience_var = tk.StringVar()
        self.skills_var = tk.StringVar()

        self.create_widgets()

    def create_widgets(self):
        # Nhãn và các trường nhập liệu
        ttk.Label(self.root, text="Vai trò:").grid(row=0, column=0, sticky=tk.W, padx=10, pady=5)
        ttk.Entry(self.root, textvariable=self.roles_var).grid(row=0, column=1, sticky=tk.EW, padx=10, pady=5)
        
        ttk.Label(self.root, text="Công Ty:").grid(row=1, column=0, sticky=tk.W, padx=10, pady=5)
        ttk.Entry(self.root, textvariable=self.companies_var).grid(row=1, column=1, sticky=tk.EW, padx=10, pady=5)
        
        ttk.Label(self.root, text="Vị trí:").grid(row=2, column=0, sticky=tk.W, padx=10, pady=5)
        ttk.Entry(self.root, textvariable=self.locations_var).grid(row=2, column=1, sticky=tk.EW, padx=10, pady=5)
        
        ttk.Label(self.root, text="Kinh nghiệm:").grid(row=3, column=0, sticky=tk.W, padx=10, pady=5)
        ttk.Entry(self.root, textvariable=self.experience_var).grid(row=3, column=1, sticky=tk.EW, padx=10, pady=5)
        
        ttk.Label(self.root, text="Kỹ năng:").grid(row=4, column=0, sticky=tk.W, padx=10, pady=5)
        ttk.Entry(self.root, textvariable=self.skills_var).grid(row=4, column=1, sticky=tk.EW, padx=10, pady=5)
        
        # Nút bấm và khung hiển thị kết quả
        ttk.Button(self.root, text="Đề xuất công việc", command=self.recommend_jobs).grid(row=5, column=0, columnspan=2, pady=10)
        self.results = tk.Text(self.root, height=10, width=50)
        self.results.grid(row=6, column=0, columnspan=2, sticky=tk.NSEW, padx=10, pady=5)

        # Cấu hình grid để khung kết quả mở rộng khi cửa sổ được phóng to
        self.root.grid_rowconfigure(6, weight=1)
        self.root.grid_columnconfigure(1, weight=1)

    def recommend_jobs(self): # Lấy dữ liệu từ các trường nhập liệu và tạo thành một câu truy vấn
        query = " ".join([self.roles_var.get(), self.companies_var.get(), self.locations_var.get(), self.experience_var.get(), self.skills_var.get()])
        recommendations = recommend_jobs(query)# Lấy đề xuất công việc từ mô hình
        self.results.delete(1.0, tk.END)# Xóa nội dung cũ trong khung kết quả
        for i, job in recommendations.iterrows(): # Chèn các kết quả mới vào khung kết quả
            self.results.insert(tk.END, f"{job['roles']} tại công ty {job['companies']} ở địa điểm {job['locations']} với kinh nghiệm {job['experience']} năm và có kỹ năng: {job['skills']}\n\n")

if __name__ == "__main__":
    root = tk.Tk()
    app = JobRecommenderApp(root)
    root.mainloop()
